## 3. A grading assistant
Teachers in general have a lot of administrations to do and one of those things is grading. Can we create a simple grade assistant to assist a Swedish teacher in grading? This exercise focuses a lot in prompt engineering and afterwards to postprocess the output using Pydantic.

a) Go into this page with examples of students answers to a particular question. Copy some example texts and paste it into files with names like student_text_1.txt, student_text_2.txt.

b) Read these data into python and tell your LLM to grade them.

c) Prompt to get an output of fields proposed_grade, motivation and improvements.

d) Now validate this with pydantic model

e) Output a folder with the following txt files: proposed_grade.txt, motivation.txt and improvements.txt

f) Go into skolverket for Svenska 1 and copy "Betygskriterier" for "Svenska 1". These are the criterias for the different grades. Paste this into a file called criterias.txt.

g) Now repeat b)-e) but with the criterias in your prompt as well. Can you see any differences in the outputs?

h) Can you improve the output quality by providing few shot examples?

a) Go into this page with examples of students answers to a particular question. Copy some example texts and paste it into files with names like student_text_1.txt, student_text_2.txt.

In [16]:
from google import genai
from pydantic import BaseModel


In [17]:
client = genai.Client()

In [19]:
# list_of_texts = []
with open("student_text_2.txt", "r", encoding = "utf-8") as file:
    student_text = file.read()
    list_of_texts.append(student_text)
list_of_texts[1]

'Inför allas blickar\nAtt ha muntliga presentationer är något som uppleves skrämmande i dagens\nsamhälle för att vi lever i ett samhälle där man oftast inte blir tvingade till att\ngöra något. Muntliga presentationer är något man, i de flesta fall brukar bli\ntvungen att göra i skolan först. Det händer när man är i den åldern då man\ntänker väldigt mycket på vad ens vänner tycker om en. Så att behöva stå framför\nde och presentera något kan kännas skrämmande för då tror man att de som\ntittar på dig dömer dig. Vilket blir något som fastar med en även när man blir\näldre. Muntliga presentationer är inte något man gör i skolan lika ofta som att\nlämna in en uppgift så man blir aldrig riktigt van med det. Att bli van med att\ntala framför andra är ett bra sätt att bota talängslan. Desto mer man pratar inför\nen grupp människor ju mer inser man att det inte är så farligt och man kommer\nsäkert komma på några knep på att kunna hantera sina nerver om man börjar att\nprata inför andra mycket 

In [20]:
text = {"text1": list_of_texts[0], "text2": list_of_texts[1]}

In [21]:
grades = []
for text in list_of_texts:
    response = client.models.generate_content(
        model = "gemini-2.5-flash",
        contents = f"""You are a swedish teacher. Rate this text: {text}. Give it a rating between 1-10."""
    )
    print(response.text)
    grades.append(response.text)




Hej! Tack för din text. Den handlar om ett intressant och relevant ämne som många kan relatera till. Du har en tydlig struktur i din text, börjar med personliga reflektioner och kopplar sedan det till en extern källa, vilket är ett bra sätt att bygga upp ett resonemang. Dina argument är logiska och lätta att följa, och du uttrycker dina åsikter väl.

Dina största utmaningar ligger i grammatiken och språkkorrektheten. Här är några områden att fokusera på:

*   **Sammansatta ord:** Detta är det mest återkommande felet. Många ord som du har skrivit isär ska sitta ihop. Exempel: "andledningarna" -> "anledningarna", "tal rädd" -> "talrädd", "klass rummet" -> "klassrummet", "jobb skäl" -> "jobbskäl", "själv förtroende" -> "självförtroende". En bra tumregel är att om två ord beskriver en sak, ska de oftast skrivas ihop i svenskan.
*   **Pronomen (de/dem/dom):** Du blandar ihop "de" och "dem" ("dom" ska dessutom aldrig användas i formellt skriftspråk). Kom ihåg att "de" är subjektsform (som "t

In [32]:
# c) Prompt to get an output of fields proposed_grade, motivation and improvements.
from typing import Literal
for text in list_of_texts:
    response = client.models.generate_content(
        model = "gemini-2.5-flash",
        contents = f"""You are a swedish teacher. Rate this text: {text}. Give me the following fields:
            proposed_grade: A - F. A being the best grade and F being the worst
            motivation: motivate the grade given to the student
            improvements: what they can improve
            Now respond with ONLY the JSON object, no extra text, no explanation. and no ```json at the start and no ``` at the end
            """
    )
print(response.text.strip("```json"))
grades.append(response.text)

ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash\nPlease retry in 27.270374374s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.

In [23]:
# d) Now validate this with pydantic model
from pydantic import BaseModel, Field
class GradingAssistant(BaseModel):
    proposed_grade: str = Field(description="Betyg för text. Möjliga betyg är [A, B, C, D, E, F]. Där A är högsta möjliga och F är det sämsta")
    motivation: str = Field(description= "Motivera varför du satt betyget")
    improvements: str = Field(description="Förklara vad som kan förbättras")

client = genai.Client()
graded = []
for text in list_of_texts:
    response = client.models.generate_content(
        model= "gemini-2.5-flash",
        contents = f"""You are a swedish teacher. Grade this text: {text}.
        """,
        config={"response_mime_type": "application/json", "response_schema": GradingAssistant}
    )
    graded.append(response.text)


In [26]:
list_of_texts[0]

'Inför allas blickar\nEn av de större andledningarna att man är tal rädd är förmodligen att man är\nrädd för att folket som lyssnar på kommer göra narr av en. När man är ensam\npå en scen eller i klass rummet så är det väldigt jobbigt när det sitter ett flertal\nmänniskor som sitter och tittar på en. Jag känner mig inte så säker när jag står och\ngör en föreläsning eller en presentation. Känslan när man är framför människor\nsom man inte riktigt känner är ganska obehagligt, för man vill inte att dem ska\ntycka illa om en, eller tycka att man är konstig på något vis. Men jag tror att det\ngår att bli av med talrädslan, om man börjar i en mindre grupp och gör sig vann\nmed att prata med människor utan att bli rädd eller känna obehag.\nPeter Letmark skriver i Dagens Nyheter 2012-05-04 att det har att göra med\nevolutionära förklaringar, att man är rädd för ormar eller hundar kan ju vara\nen sak som sitter i ryggmärgen. Det kan bero på att man är rädd för att bli\nutstöt från gruppen. Letm

In [24]:
for grade in graded:
    print(grade)

{
  "proposed_grade": "D",
  "motivation": "Texten behandlar ett relevant ämne och har en begriplig struktur där personliga reflektioner varvas med en extern källa. Argumentationen är tydlig och författaren lyckas förmedla sina tankar om talrädsla och hur den kan övervinnas. Innehållsmässigt är texten godkänd.",
  "improvements": "Språket innehåller ett stort antal grammatiska fel, stavfel och särskrivningar (t.ex. 'tal rädd', 'klass rummet', 'jobb skäl', 'själv förtroende'). Det finns också felaktiga ordval ('andledningarna' istället för 'anledningarna', 'gör sig vann' istället för 'vänjer sig') samt problem med syftning och meningsbyggnad (t.ex. 'utan att inte veta', 'blir du väldigt osäker'). Användningen av 'dom' istället för 'de' är ett återkommande fel. Dessa språkliga brister gör texten svårläst och drar ner helhetsintrycket avsevärt. Ett stort fokus bör läggas på grundläggande svensk grammatik, stavning av sammansatta ord och att konstruera tydligare och mer korrekta meningar."